In [155]:
from math import nan
import pandas as pd
from charset_normalizer import from_path 
import re 
from openpyxl import load_workbook 
import debugpy


In [156]:
colunas_df_posicao = ['table', 'info','label', 'line', 'column']
df_posicao = pd.DataFrame(columns=colunas_df_posicao)
df_posicao = df_posicao.set_index(['table', 'info', 'label'])

Função para busca a posição do dado na planilha 

In [157]:
def find_pos(df_posicao, table, info, label):
    try:
        filtro = (
            (df_posicao.index.get_level_values('table') == table) &
            (df_posicao.index.get_level_values('info') == info) &
            (df_posicao.index.get_level_values('label') == label)
        )

        if not filtro.any():
            print(f"[AVISO] Registro não encontrado: ({table}, {info}, {label})")
            return None, None

        linha = df_posicao.loc[filtro, 'line'].iloc[0]
        coluna = df_posicao.loc[filtro, 'column'].iloc[0]

        return linha, coluna

    except Exception as e:
        print(f"[ERRO] Falha ao localizar posição: {e}")
        return None, None

In [158]:


config_path = r"C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\doc"
config_file = r"\eda.xlsx"
sheet_file = config_path+config_file
df_eda_sheet = pd.read_excel(sheet_file,sheet_name='tabelas',header=None)

# print(df_eda_sheet.loc[0,0])

# cia o dataframe de tabelas 
# tabelas = df_eda_sheet[0]
# tabelas = tabelas[tabelas.notna()]   
# df_tabelas = tabelas.to_frame()

df_tabelas = {}


Montagem do dataframe com dados das tabelas

In [159]:
# Localiza as celulas com os dados das tabelas 
col1 = df_eda_sheet[1]
col1 = col1[col1 == "table"]
    #tab_names.append(col1)

# Loop nas linhas e colunas com dados de tabelas e criação do dataframe
labels, values, dict_lst, posicoes = [],[],[],[]
tab_names = [] 
for idx, txt in col1.items(): 
    line = idx+1
    col = 1
    while not pd.isna(df_eda_sheet.loc[line,col]):
        label = df_eda_sheet.loc[line,col]
        line_value = line+1 # correção da linha pra busca futura com o openpyxl
        value = df_eda_sheet.loc[line_value,col]
        if label == "name": 
            table_name = value
            tab_names.append(table_name)
        labels.append(label)
        values.append(value) 
        # Guarda a posição da celula
        df_posicao.loc[(table_name, 'tabela', label.lower()), ['line', 'column']] = [line_value + 1, col + 1]
        col += 1 # Proxima coluna
    # Monta o dicionario temporario com os labels e valores 
    temp_dict = dict(zip(labels, values))
    # adiciona o registro do campo pra criação do dataset no final
    dict_lst.append(temp_dict)
# Cria o dataset de dados de tabelas
df_tabelas = pd.DataFrame(dict_lst)
# Padroniza os labels como 'minúsculo'
df_tabelas.columns = df_tabelas.columns.str.lower() 


Montagem do dataframe com dados de campos 


In [ ]:
# Localiza as celulas com os dados dos campos
col1 = df_eda_sheet[1]
col1 = col1[col1 == "fields"]
tab_idx = 0 
line_base_pointer = 1
col_base_pointer = 1 
# Loop nas linhas e colunas com dados de campos e criação do dataframe
labels, labels_lst, values, dict_lst, posicoes = [],[],[],[],[]

# Monta a lista de labels
idx = col1.index[0]
line = idx+1
col = 1  
while not pd.isna(df_eda_sheet.loc[line,col]):
    label = df_eda_sheet.loc[line,col]
    labels_lst.append(label)
    col += 1 # Proxima coluna

# Loop nos valores
line = 0 
table_idx = 0 
for idx, txt in col1.items():
    col = 1         
    line = idx + 2  
    while not pd.isna(df_eda_sheet.loc[line,1]): 
        table_name = tab_names[tab_idx]
        try:
            label = labels_lst[col-1]
            labels.append(label)
            value = df_eda_sheet.loc[line,col]
            values.append(value) 
            # Guarda a posição da celula
            df_posicao.loc[(table_name, 'field', label.lower()), ['line', 'column']] = [line_value + 1, col + 1]            
            col = col + 1                     
        except IndexError:
            # Monta o dicionario temporario com os labels e valores 
            temp_dict = dict(zip(labels, values))
            # adiciona o registro do campo pra criação do dataset no final
            dict_lst.append(temp_dict)
            labels, values = [], [] 
            col = 1
            line = line + 1
            if line == len(df_eda_sheet): 
                break
    tab_idx += 1 # Proxima tabela 
    
# Cria o dataset de dados de tabelas
df_campos = pd.DataFrame(dict_lst)
# Padroniza os labels como 'minúsculo'
df_campos.columns = df_campos.columns.str.lower() 

print(df_campos)


#     while True:
#         col = 1 
#         if line_base_pointer == 1: 
#             line = idx+line_base_pointer
#         else: 
#             line += line_base_pointer
#         if pd.isna(df_eda_sheet.loc[line,col]): # Verifica se a proxima linha possui dados
#             break  
#         table_name = tab_names[tab_idx]
#         # Percorre as colunas dos atributos horizontalmente até encontrar null 
#         while not pd.isna(df_eda_sheet.loc[line,col]):
#             if line_base_pointer == 1: 
#                 label = df_eda_sheet.loc[line,col]
#                 labels.append(label)
#             else: 
#                 label = labels[col-1]
#             line_value = line
#             value = df_eda_sheet.loc[line_value,col]
#             values.append(value) 
#             # Guarda a posição da celula
#             df_posicao.loc[(table_name, 'field', label.lower()), ['line', 'column']] = [line_value + 1, col + 1]
#             col += 1 # Proxima coluna
#         # Move o ponteiro para a proxima linha 
#         line_base_pointer += 1 

#     tab_idx += 1 # Proxima tabela
#     # Monta o dicionario temporario com os labels e valores 
#     temp_dict = dict(zip(labels, values))
#     # adiciona o registro do campo pra criação do dataset no final
#     dict_lst.append(temp_dict)
# # Cria o dataset de dados de tabelas
# df_campos = pd.DataFrame(dict_lst)
# # Padroniza os labels como 'minúsculo'
# df_campos.columns = df_campos.columns.str.lower() 

#print(df_campos)

                 name                     description   type  size  format
0              coenti  Código da empresas seguradoras    int   NaN     NaN
1              noenti                 Nome da empresa    str   NaN     NaN
2              coramo                  Código do ramo    int   NaN     NaN
3              noramo                   Nome dro ramo    str   NaN     NaN
4              coenti               Código da Empresa    int   NaN     NaN
5              coramo           Código do Ramo no FIP    int   NaN     NaN
6            damesano         Ano e mês da informação    int   NaN  YYYYMM
7       premio_direto              Prêmio Direto (R$)  float   NaN     NaN
8        premio_ganho               Prêmio Ganho (R$)  float   NaN     NaN
9   premio_de_seguros             Prêmio Seguros (R$)  float   NaN     NaN
10    sinistro_direto        Sinistro de Seguros (R$)  float   NaN     NaN
11    sinistro_retido            Sinistro Retido (R$)  float   NaN     NaN
12           desp_com    

In [161]:
# # teste de atribuição de valor a uma celula salva
# # Abre novamente a planilha para salvamento
# try:
#     workbook = load_workbook(sheet_file)
# except FileNotFoundError:
#     print("Erro: Arquivo 'seu_arquivo.xlsx' não encontrado.")
# sheet = workbook['tabelas'] 
# linha, coluna = find_pos(df_posicao, 'ses_ramos', 'tabela', 'motivation')
# sheet.cell(row=linha, column=coluna, value='Funfou!')
# workbook.save(r'C:\Users\User\OneDrive\Documentos\GitHub\analise-dados-susep-seguros\doc\eda.xlsx')


Montagem do dataframe com dados dos campos 